# One Portfolio to Rule Them All!

This notebook deals with the collection of financial asset data (stocks and cryptocurrency). This must be run in an environment that is compatible with alpacas API.

1. Collect data from APIs
2. Save to csv
3. Collect all data in SQL Database

In [1]:
# Import all necessary libraries
import os
from pathlib import Path
from dotenv import load_dotenv
import pandas as pd
import alpaca_trade_api as tradeapi
import datetime
from dateutil.relativedelta import relativedelta

## Collect Data From APIs

In [2]:
# Import data from API/csv into Pandas DataFrame
load_dotenv()

# Set Alpaca API key and secret
alpaca_api_key=os.getenv('APCA_API_KEY_ID')
alpaca_secret_key=os.getenv('APCA_API_SECRET_KEY')
alpaca_endpoint=os.getenv('APCA_API_BASE_URL')

# Create the Alpaca API object
alpaca = tradeapi.REST()


# *** IN GENERAL ***
# we could parameterize the dates...

# Format current and previous date as ISO format
date = datetime.date.today()
date_fmt = date.strftime('%Y-%m-%d')
today = pd.Timestamp(date_fmt, tz='America/New_York').isoformat()

# Set start date of five years back from today.
# Sample results may vary from the solution based on the time frame chosen
five_yrs_ago = date - relativedelta(years=5)
five_yrs_ago = five_yrs_ago.strftime('%Y-%m-%d')
start_date = pd.Timestamp(five_yrs_ago, tz='America/New_York').isoformat()

# Set the tickers
tickers = ['AAPL', 'MSFT', 'PFE', 'DIS']

# Set timeframe to "1Day" for Alpaca API
timeframe = "1Day"

# Get current closing prices for SPY and AGG
# The current day may be a day when the markets are closed (weekend, holiday, etc.)
# So, if the retrieved portfolio is empty, let's try the previous day.
tickers_df = alpaca.get_bars(tickers, timeframe, start=start_date, end=today).df

while tickers_df.empty:
    date -= relativedelta(days=1)
    date_fmt = date.strftime('%Y-%m-%d')
    today = pd.Timestamp(date_fmt, tz='America/New_York').isoformat()
    tickers_df = alpaca.get_bars(tickers, timeframe, start=start_date, end=today).df

# tickers_df.set_index(['timestamp'])
tickers_df.head()

,open,high,low,close,volume,trade_count,vwap,symbol
timestamp,,,,,,,,
2018-04-13 04:00:00+00:00,174.7800,175.8400,173.8500,174.64,25127526,156694,174.861677,AAPL
2018-04-16 04:00:00+00:00,175.0301,176.1900,174.8301,175.82,21579320,155498,175.646643,AAPL
2018-04-17 04:00:00+00:00,176.4900,178.9365,176.4100,178.24,26605711,168081,177.876089,AAPL
2018-04-18 04:00:00+00:00,177.8100,178.8200,176.8800,177.84,20757396,136339,177.949113,AAPL
2018-04-19 04:00:00+00:00,174.9600,175.3900,172.6600,172.80,34822915,230448,173.639715,AAPL


In [3]:
# Review and clean data
tickers_df_list = [tickers_df[tickers_df['symbol'] == ticker].drop('symbol', axis='columns') for ticker in tickers]
stocks_df = pd.concat(tickers_df_list, axis='columns', join='inner', keys=tickers).dropna()
display(stocks_df.head())
display(stocks_df.tail())

AAPL                                        \
                               open      high       low   close    volume   
timestamp                                                                   
2018-04-13 04:00:00+00:00  174.7800  175.8400  173.8500  174.64  25127526   
2018-04-16 04:00:00+00:00  175.0301  176.1900  174.8301  175.82  21579320   
2018-04-17 04:00:00+00:00  176.4900  178.9365  176.4100  178.24  26605711   
2018-04-18 04:00:00+00:00  177.8100  178.8200  176.8800  177.84  20757396   
2018-04-19 04:00:00+00:00  174.9600  175.3900  172.6600  172.80  34822915   

                                                    MSFT                ...  \
                          trade_count        vwap   open   high    low  ...   
timestamp                                                               ...   
2018-04-13 04:00:00+00:00      156694  174.861677  94.05  94.18  92.44  ...   
2018-04-16 04:00:00+00:00      155498  175.646643  94.07  94.66  93.42  ...   
2018-04-17 04:00:00+00:00      168081  177.876089  95.00  96.54  94.88  ...   
2018-04-18 04:00:00+00:00      136339  177.949113  96.22  96.72  95.52  ...   
2018-04-19 04:00:00+00:00      230448  173.639715  96.44  97.07  95.34  ...   

                                PFE                            DIS            \
                             volume trade_count       vwap    open      high   
timestamp                                                                      
2018-04-13 04:00:00+00:00  16864266       62505  36.328367  101.00  101.5200   
2018-04-16 04:00:00+00:00  15116082       68440  36.552518  100.69  101.0000   
2018-04-17 04:00:00+00:00  16769973       69431  36.388082  101.20  102.5900   
2018-04-18 04:00:00+00:00  13905179       59911  36.472182  102.67  102.7100   
2018-04-19 04:00:00+00:00  15784274       72763  36.478912  101.00  101.5799   

                                                                             
                              low   close    volume trade_count        vwap  
timestamp                                                                    
2018-04-13 04:00:00+00:00  100.16  100.35   6324606       43426  100.572233  
2018-04-16 04:00:00+00:00   99.73  100.24  10328002       51573  100.166105  
2018-04-17 04:00:00+00:00  100.75  102.17   9727561       58888  101.947372  
2018-04-18 04:00:00+00:00  101.20  101.21   5852898       43325  101.757937  
2018-04-19 04:00:00+00:00  100.22  100.89   6804490       47981  100.788706  

[5 rows x 28 columns]

AAPL                                      \
                             open      high     low   close    volume   
timestamp                                                               
2023-04-06 04:00:00+00:00  162.43  164.9584  162.00  164.66  45390035   
2023-04-10 04:00:00+00:00  161.42  162.0300  160.08  162.03  47791508   
2023-04-11 04:00:00+00:00  162.35  162.3600  160.51  160.80  47793761   
2023-04-12 04:00:00+00:00  161.22  162.0600  159.78  160.10  45152271   
2023-04-13 04:00:00+00:00  161.63  165.8000  161.42  165.56  67382278   

                                                      MSFT                  \
                          trade_count        vwap     open    high     low   
timestamp                                                                    
2023-04-06 04:00:00+00:00      446212  164.025748  283.210  292.08  282.03   
2023-04-10 04:00:00+00:00      564828  161.261844  289.208  289.60  284.71   
2023-04-11 04:00:00+00:00      490480  161.128148  285.750  285.98  281.64   
2023-04-12 04:00:00+00:00      498300  160.947343  284.790  287.01  281.96   
2023-04-13 04:00:00+00:00      585134  164.351669  283.590  289.90  283.17   

                           ...       PFE                            DIS  \
                           ...    volume trade_count       vwap    open   
timestamp                  ...                                            
2023-04-06 04:00:00+00:00  ...  25931833      131406  41.568939   99.44   
2023-04-10 04:00:00+00:00  ...  15138845       98244  41.537303   99.30   
2023-04-11 04:00:00+00:00  ...  15304123       88991  41.847967  101.16   
2023-04-12 04:00:00+00:00  ...  16376449       90758  41.480120  101.25   
2023-04-13 04:00:00+00:00  ...  22396886      111275  41.519420   98.51   

                                                                        \
                             high     low   close   volume trade_count   
timestamp                                                                
2023-04-06 04:00:00+00:00  100.32   98.55   99.97  7042486       91244   
2023-04-10 04:00:00+00:00  100.81   98.90  100.81  8019029      116458   
2023-04-11 04:00:00+00:00  101.91  100.29  100.42  7504624      102404   
2023-04-12 04:00:00+00:00  102.22   97.70   97.94  8968871      113998   
2023-04-13 04:00:00+00:00  101.07   98.51  100.84  8597227       93757   

                                       
                                 vwap  
timestamp                              
2023-04-06 04:00:00+00:00   99.608163  
2023-04-10 04:00:00+00:00  100.275738  
2023-04-11 04:00:00+00:00  101.033819  
2023-04-12 04:00:00+00:00   99.334797  
2023-04-13 04:00:00+00:00  100.426484  

[5 rows x 28 columns]

In [4]:
# save to csv
output_file = Path('./stocks_data.csv')
stocks_df.to_csv(output_file)